# Cours TAL – Labo 7 : Classification de dépêches d’agence avec NLTK

**Objectifs**
L’objectif de ce labo est de réaliser des expériences de classification de documents avec la boîte à
outils NLTK sur le corpus de dépêches Reuters. Le labo est à effectuer en binôme. Le rendu sera un
notebook Jupyter présentant vos choix, votre code, vos résultats et les discussions. Le labo sera jugé
sur la qualité des expériences et sur la discussion des différentes options explorées.


# 1. Récupération des données

**Données :** les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Veuillez respecter la
division en données d’entraînement et données de test.

In [6]:
import nltk
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer

# nltk.download('reuters')
# nltk.download('stopwords')
# nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

stop_words = set(nltk.corpus.stopwords.words('english'))

# Nous faisons le choix de traiter tous les mots en minuscule
documents = [(list(w.lower() for w in reuters.words(fileid)), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

documents_no_stop_words = [(list(w.lower() for w in filter(lambda w: w.lower() not in stop_words, reuters.words(fileid))), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

documents_lemmatized = [(list(w.lower() for w in map(lemmatizer.lemmatize, reuters.words(fileid))), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

In [36]:
# Récupère la fréquence des mots en filtrant les mots de moins de 4 caractères
import re
all_words = nltk.FreqDist(w.lower() for w in reuters.words() if re.match(r'^[a-z]{3,}$', w))
# all_words = nltk.FreqDist(w.lower() for w in reuters.words())
# On sélectionne les 2000 mots les plus fréquents comme features pour les classifieurs
word_features = list(all_words)[:2000]

In [37]:
import random

# Modifie les catégories des documents pour les rendre binaires
def documents_with_binary_category(documents, category):
    return [(d, category if c == category else 'other') for (d, c) in documents]

# split un dataset en 80% train et 20% test pour chaque category
def split_dataset(documents):
    dataset = {}
    for (d, c) in documents:
        if c not in dataset:
            dataset[c] = []
        dataset[c].append(d)
    train_set = []
    test_set = []
    for c in dataset:
        random.shuffle(dataset[c])
        train_set += [(d, c) for d in dataset[c][int(len(dataset[c]) * 0.2):]]
        test_set += [(d, c) for d in dataset[c][:int(len(dataset[c]) * 0.2)]]
    return train_set, test_set

# Retourne un dictionnaire indiquant si une feature est présent dans le document
def document_contains_features(document):
    doc_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in doc_words)
    return features

def get_featuresets(documents):
    return [(document_contains_features(d), c) for (d,c) in documents]

# def train_classifier(documents):
    # Mélange des documents pour ne pas récupérer les mêmes données de tests à chaque exécution
    # random.shuffle(documents) 

    # featuresets = [(document_contains_features(d), c) for (d,c) in documents]
    
    # train_set, test_set = split_dataset(featuresets)
    
    # print la taille des sets pour chaque catégorie
    # print('Train set size of money-fx:', len(list(filter(lambda x: x[1] == 'money-fx', train_set))))
    # print('Train set size of other:', len(list(filter(lambda x: x[1] == 'other', train_set))))
    # print('Test set size:', len(list(filter(lambda x: x[1] == 'money-fx', test_set))))
    # print('Test set size:', len(list(filter(lambda x: x[1] == 'other', test_set))))

    # classifier = nltk.NaiveBayesClassifier.train(train_set)
    # print(nltk.classify.accuracy(classifier, test_set))
    # classifier.show_most_informative_features(5)
    # return classifier

## 2. Classifieurs binaires

### 2.1 Classifieur binaire pour la catégorie 'money-fx'

#### 2.1.1 Classifieur Bayésien naïf + lemmatisation

In [38]:
dataset_moneyfx_lemmatized = documents_with_binary_category(documents_lemmatized, 'money-fx')

train_set_moneyfx_lemmatized, test_set_moneyfx_lemmatized = split_dataset(get_featuresets(dataset_moneyfx_lemmatized))

classifier_moneyfx_lemmatized = nltk.NaiveBayesClassifier.train(train_set_moneyfx_lemmatized)

In [ ]:
# calcul de la précision, du rappel et du f-score de classifier_moneyfx_lemmatized
from nltk.metrics import precision, recall, f_measure

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set_moneyfx_lemmatized):
    refsets[label].add(i)
    observed = classifier_moneyfx_lemmatized.classify(feats)
    testsets[observed].add(i)
    
print('money-fx')

print('Precision:', precision(refsets['money-fx'], testsets['money-fx']))
print('Recall:', recall(refsets['money-fx'], testsets['money-fx']))
print('F-Score:', f_measure(refsets['money-fx'], testsets['money-fx']))

classifier_moneyfx_lemmatized.show_most_informative_features(5)